## Lecture 8: Eigen Faces

In [1]:
import numpy as np
import os

import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.image import imread

plt.rcParams['xtick.labelsize']=20      # change the tick label size for x axis
plt.rcParams['ytick.labelsize']=20      # change the tick label size for x axis
plt.rcParams['axes.linewidth']=1        # change the line width of the axis
plt.rcParams['xtick.major.width'] = 3   # change the tick line width of x axis
plt.rcParams['ytick.major.width'] = 3   # change the tick line width of y axis
rc('text', usetex=False)                # disable LaTeX rendering in plots
rc('font',**{'family':'DejaVu Sans'})   # set the font of the plot to be DejaVu Sans

### 0. Mount Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/MyDrive/ME491"
data_path = os.path.join(path, "data/allFaces.mat")

Mounted at /content/drive


### 1. Load Face Image Data

In [3]:
import scipy.io

mat_contents = scipy.io.loadmat(data_path)

When encounter a large dataset, the first to do is to look at it, and have a general understanding of what kind of variables are in it.

In [ ]:
for key, value in mat_contents.items():
  print(key)

In [ ]:
faces = mat_contents['faces']
n = int(mat_contents['n']) # number of rows for each face
m = int(mat_contents['m']) # number of columns for each face
nfaces = np.ndarray.flatten(mat_contents['nfaces']) # number of pictures for each person

print(nfaces)
print("Shape of faces is:", faces.shape)
print(sum(nfaces))
print("Shape of images is:", n, m)

In [ ]:
plt.imshow(faces, cmap = 'Greys_r')
plt.axis('off')

### 2. Plot All Faces

In [ ]:
# Now we want to plot the first image of the first 36 person
allPersons = np.zeros((n*6,m*6))
count = 0

for j in range(6):
  for k in range(6):
    allPersons[j*n:(j+1)*n, k*m:(k+1)*m] = np.reshape(faces[:,np.sum(nfaces[:count])],(m,n)).T
    count += 1

img = plt.imshow(allPersons)
img.set_cmap('gray')
plt.axis('off')

To practice playing with `numpy` arrays, let's now plot all faces for a given person

In [ ]:
idx = 37 # define which person we want to plot
num_pic = nfaces[idx] # find out the number of pictures (different for every person)
i = 8 # since number of pictures are around 64, we will have 8 rows
j = int(np.ceil(num_pic/i)) # columns is defined by the number of pictures

allPhotos = np.zeros((n*i, m*j))
count = 0

for ii in range(i):
  for jj in range(j):
    if count >= num_pic:
      break
    else:
      allPhotos[ii*n:(ii+1)*n, jj*m:(jj+1)*m] = np.reshape(faces[:,int(np.sum(nfaces[:idx]))+count],(m,n)).T
      count += 1

img = plt.imshow(allPhotos, cmap = "Greys_r")
plt.axis('off')

### 3. Compute Eigen Face

In [ ]:
# We use the first 36 people for training data
trainingFaces = faces[:,:np.sum(nfaces[:36])]
avgFace = np.mean(trainingFaces,axis=1) # size n*m by 1

# Let's look at the average face
plt.imshow(np.reshape(avgFace, (m,n)).T, cmap="Greys_r")
plt.axis('off')

Note: the only time features are every column is in Ch. 1.5, here, features are in rows while every column is a new snapshot, so instead of the V matrix being the principal components, U is the matrix with all the principal components.  In this case, we call them the eigenfaces.

In [7]:
# Compute eigenfaces on mean-subtracted training data
X = trainingFaces - np.tile(avgFace,(trainingFaces.shape[1],1)).T
U, S, VT = np.linalg.svd(X,full_matrices=0)

In [15]:
print(np.max(U[:,0]))

0.0014828167834332656


In [ ]:
# Draw the first eigen face
plt.imshow(np.reshape(U[:,1],(m,n)).T, vmin = -1e-2, vmax = 1e-2, cmap="coolwarm")
plt.axis("off")

In [ ]:
# Now Let's plot the first 10 eigen faces

i = 2
j = 5

eigenfaces = np.zeros((n*i, m*j))
count = 0

for ii in range(i):
  for jj in range(j):
    eigenfaces[ii*n:(ii+1)*n, jj*m:(jj+1)*m] = np.reshape(U[:,count],(m,n)).T
    count += 1

img = plt.imshow(eigenfaces, vmin = -1e-2, vmax = 1e-2, cmap="coolwarm")
plt.axis('off')

In [ ]:
# Let's now look at the singular values
idx = np.where(S > 1.0)
print(len(S))
print(len(idx[0]))
plt.semilogy(S[idx], 'o')

### 4. Represent Person 37 with Eigen faces

In [ ]:
testFace = faces[:,np.sum(nfaces[:36])] # Person 37
testFaceMS = testFace - avgFace
r_list = [25, 50, 100, 200, 400, 800, 1600]
r = 1600
reconFace = avgFace + U[:,:r] @ (U[:,:r].T @ testFaceMS)
img = plt.imshow(np.reshape(reconFace,(m,n)).T, cmap="Greys_r")
plt.axis("off")

### 5. principal component projection

In [22]:
P1num = 2 # Person number 2
P2num = 7 # Person number 7
P1 = faces[:,np.sum(nfaces[:(P1num-1)]):np.sum(nfaces[:P1num ])]
P2 = faces[:,np.sum(nfaces[:(P2num-1)]):np.sum(nfaces[:P2num ])]
P1 = P1 - np.tile(avgFace,(P1.shape[1],1)).T
P2 = P2 - np.tile(avgFace,(P2.shape[1],1)).T
PCAmodes = [5, 6] # Project onto PCA modes 5 and 6
PCACoordsP1 = U[:,PCAmodes-np.ones_like(PCAmodes)].T @ P1
PCACoordsP2 = U[:,PCAmodes-np.ones_like(PCAmodes)].T @ P2

In [ ]:
plt.plot(PCACoordsP1[0,:],PCACoordsP1[1,:],'d',c='k')
plt.plot(PCACoordsP2[0,:],PCACoordsP2[1,:],'^',c='r')
# plt.xlim([-500, 500])
# plt.ylim([-500, 500])